In [26]:
import numpy as np
import pandas as pd
import seaborn as sns
pd.set_option('display.max_columns', 100) # Setting pandas to display a N number of columns
pd.set_option('display.max_rows', 10) # Setting pandas to display a N number rows
pd.set_option('display.width', 100) # Setting pandas dataframe display width to N
import matplotlib.pyplot as plt # plotting library
from src.utils import import_data, categorial_feature_overview, reduce_mem_usage, make_mi_scores, plot_mi_scores

import ppscore as pps # importing ppscore
import numpy as np
import pandas as pd
import seaborn as sns
from xgboost import plot_importance, XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
import pickle
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split, StratifiedKFold
from src.utils import preprocess
%matplotlib inline

In [13]:
file = "usersessions/train.json"
df = pd.read_csv("train_processed.csv")
df = reduce_mem_usage(df)


Memory usage of dataframe is 12.06 MB
Memory usage after optimization is: 8.90 MB
Decreased by 26.2%


In [19]:
293/60_000 * 100

0.48833333333333334

In [18]:
103/20_000 * 100

0.515

In [3]:
df[["dayofweek", "day", "month", "year"]] = df[
    ["dayofweek", "day", "month", "year"]
].astype(str)


In [4]:
useful_cols = df.drop(["time", "date_hour"], axis=1).columns
df = df[useful_cols]
cat_cols = df.select_dtypes(include=["object"]).columns
num_cols = df.select_dtypes(include=["number"]).columns



In [6]:
enc = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)

enc.fit(df[cat_cols])
# pickle.dump(enc.fit(df[cat_cols]), open("encoder.pkl", 'wb'))

df[cat_cols] = enc.transform(df[cat_cols])

In [7]:
X = df.drop("is_joe", axis=1)
y = df["is_joe"]

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y.values, test_size=0.25, random_state=1, stratify=y
)


In [9]:
base_model = XGBClassifier()
base_model.fit(X_train, y_train)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/xgboost/sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:34:53] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [10]:
from sklearn.metrics import (
    roc_auc_score,
    f1_score,
    classification_report,
)

predictions = base_model.predict(X_test)
print("ROC: " + str(roc_auc_score(y_test, predictions)))
print("F1: " + str(f1_score(y_test, predictions)))
print(classification_report(y_test, predictions))


ROC: 0.9756882296709107
F1: 0.9516129032258065
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     12688
           1       0.95      0.95      0.95        62

    accuracy                           1.00     12750
   macro avg       0.98      0.98      0.98     12750
weighted avg       1.00      1.00      1.00     12750



In [11]:
from eli5 import (
    show_weights,
    show_prediction,
    explain_weights_xgboost,
    explain_prediction,
    format_as_dataframes,
)

show_weights(base_model)


Weight,Feature
0.2027,locale
0.0917,city
0.0878,browser
0.0786,hour_fixed
0.0755,gender
0.0748,site_0
0.0692,site_4
0.0614,site_1
0.0512,country
0.0362,site_2


In [141]:
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split, StratifiedKFold
k = 5
kfold = StratifiedKFold(k)
evalMetric = 'auc' 


def objective(trial):
    param = {
        "verbosity": 0,
        "objective": "binary:logistic",
        "n_estimators": trial.suggest_int("n_estimators", 100,600,50),
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        "lambda": trial.suggest_loguniform("lambda", 1e-8, 1.0),
        "alpha": trial.suggest_loguniform("alpha", 1e-8, 1.0),
        "learning_rate": trial.suggest_uniform("learning_rate", 1e-3, 1e-1),
        "subsample_range": trial.suggest_uniform("subsample_range", 1e-3, 99e-2),
        "colsample_bytree": trial.suggest_uniform("colsample_bytree", 1e-3, 99e-2),
        "scale_pos_weight": trial.suggest_int('scale_pos_weight', 1, 100),
        "min_child_weight": trial.suggest_int('min_child_weight', 1, 20),

    }
    if param["booster"] == "gbtree" or param["booster"] == "dart":
        param["max_depth"] = trial.suggest_int("max_depth", 1, 9)
        param["eta"] = trial.suggest_loguniform("eta", 1e-8, 1.0)
        param["gamma"] = trial.suggest_loguniform("gamma", 1e-8, 1.0)
        param["grow_policy"] = trial.suggest_categorical(
            "grow_policy", ["depthwise", "lossguide"]
        )
    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical(
            "sample_type", ["uniform", "weighted"]
        )
        param["normalize_type"] = trial.suggest_categorical(
            "normalize_type", ["tree", "forest"]
        )
        param["rate_drop"] = trial.suggest_loguniform("rate_drop", 1e-8, 1.0)
        param["skip_drop"] = trial.suggest_loguniform("skip_drop", 1e-8, 1.0)

    # Callback for pruning
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation-auc")

    
    # train_x, test_x, train_y, test_y = train_test_split(
    #     X, y, test_size=0.25, stratify=y
    # )
    # dtrain = xgb.DMatrix(train_x, label=train_y)
    # dtest = xgb.DMatrix(test_x, label=test_y)
    scores = []
    for train_index, test_index in kfold.split(X, y):

        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        dtrain = xgb.DMatrix(X_train, label=y_train)
        dtest = xgb.DMatrix(X_test, label=y_test)
        bst = xgb.train(
            param, dtrain,
        )
        preds = bst.predict(dtest)
        pred_labels = np.rint(preds)
        auc = f1_score(y_test, pred_labels)
        scores.append(auc)

    return np.mean(scores)


In [142]:
def xgb_f1(y, t, threshold=0.5):
    t = t.get_label()
    y_bin = (y > threshold).astype(int) # works for both type(y) == <class 'numpy.ndarray'> and type(y) == <class 'pandas.core.series.Series'>
    return 'f1',f1_score(t,y_bin)

In [143]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=300)
print(study.best_trial)


[I 2022-01-31 18:10:55,749] A new study created in memory with name: no-name-c564e977-6b60-456d-bf64-d6f72680b56d
[I 2022-01-31 18:10:56,119] Trial 0 finished with value: 0.0 and parameters: {'n_estimators': 200, 'booster': 'gblinear', 'lambda': 2.2329495256443097e-07, 'alpha': 0.0004976962701844803, 'learning_rate': 0.09888119414016895, 'subsample_range': 0.9818662289421285, 'colsample_bytree': 0.5594142845431764, 'scale_pos_weight': 12, 'min_child_weight': 20}. Best is trial 0 with value: 0.0.
[I 2022-01-31 18:10:56,457] Trial 1 finished with value: 0.11366609344075532 and parameters: {'n_estimators': 300, 'booster': 'gblinear', 'lambda': 1.975843255185892e-07, 'alpha': 1.0016903155374291e-08, 'learning_rate': 0.0751453800192077, 'subsample_range': 0.2576865611445281, 'colsample_bytree': 0.4829556910582485, 'scale_pos_weight': 88, 'min_child_weight': 13}. Best is trial 1 with value: 0.11366609344075532.
[I 2022-01-31 18:10:56,914] Trial 2 finished with value: 0.0 and parameters: {'n_

FrozenTrial(number=294, values=[0.9072143704497672], datetime_start=datetime.datetime(2022, 1, 31, 18, 15, 48, 631047), datetime_complete=datetime.datetime(2022, 1, 31, 18, 15, 49, 671396), params={'n_estimators': 350, 'booster': 'gbtree', 'lambda': 6.996724206837135e-07, 'alpha': 6.072852734701245e-07, 'learning_rate': 0.07601386498641889, 'subsample_range': 0.4698031956026117, 'colsample_bytree': 0.3263890851217871, 'scale_pos_weight': 17, 'min_child_weight': 6, 'max_depth': 8, 'eta': 4.79311751723224e-06, 'gamma': 1.0315649795722175e-06, 'grow_policy': 'depthwise'}, distributions={'n_estimators': IntUniformDistribution(high=600, low=100, step=50), 'booster': CategoricalDistribution(choices=('gbtree', 'gblinear', 'dart')), 'lambda': LogUniformDistribution(high=1.0, low=1e-08), 'alpha': LogUniformDistribution(high=1.0, low=1e-08), 'learning_rate': UniformDistribution(high=0.1, low=0.001), 'subsample_range': UniformDistribution(high=0.99, low=0.001), 'colsample_bytree': UniformDistribu

In [144]:
params = study.best_params
params


{'n_estimators': 350,
 'booster': 'gbtree',
 'lambda': 6.996724206837135e-07,
 'alpha': 6.072852734701245e-07,
 'learning_rate': 0.07601386498641889,
 'subsample_range': 0.4698031956026117,
 'colsample_bytree': 0.3263890851217871,
 'scale_pos_weight': 17,
 'min_child_weight': 6,
 'max_depth': 8,
 'eta': 4.79311751723224e-06,
 'gamma': 1.0315649795722175e-06,
 'grow_policy': 'depthwise'}

In [145]:
from xgboost import plot_importance


In [146]:
xgb_optuna = xgb.XGBClassifier(**params)
xgb_importance = xgb_optuna.fit(X_train, y_train)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/xgboost/sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [147]:
show_weights(xgb_optuna)

Weight,Feature
0.1392,site_3
0.1054,locale
0.0844,site_0
0.0818,gender
0.0585,site_9
0.0567,site_2
0.0541,hour_fixed
0.0514,site_1
0.0443,site_10
0.0375,site_11


In [148]:
kf = StratifiedKFold(5)
scores = []

xgb_optuna = xgb.XGBClassifier(use_label_encoder=False, **params)

for train_index, test_index in kf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    xgb_optuna.fit(X_train, y_train)
    preds = xgb_optuna.predict(X_test)
    pred_labels = np.rint(preds)
    auc = roc_auc_score(y_test, pred_labels)
    scores.append(auc)
    print(auc)
print("*" * 50)

print(np.mean(scores))


0.9285714285714286
0.98
0.9198029556650247
0.9696551724137931
0.929950738916256
**************************************************
0.9455960591133005


In [150]:
print(classification_report(y_test, preds))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00     10150
           1       0.95      0.82      0.88        50

    accuracy                           1.00     10200
   macro avg       0.98      0.91      0.94     10200
weighted avg       1.00      1.00      1.00     10200



In [155]:
model = XGBClassifier(**params, use_label_encoder=False)
filename = "model.pkl"
model.fit(X, y)
# pickle.dump(model, open(filename, 'wb'))


In [149]:
kf = StratifiedKFold(5)
scores = []

xgb_xd = xgb.XGBClassifier(use_label_encoder=False)

for train_index, test_index in kf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    xgb_xd.fit(X_train, y_train)
    preds = xgb_xd.predict(X_test)
    pred_labels = np.rint(preds)
    auc = roc_auc_score(y_test, pred_labels)
    scores.append(auc)
    print(auc)
print("*" * 50)

print(np.mean(scores))


0.9183673469387755
0.97
0.8998029556650247
0.9598522167487683
0.9099014778325123
**************************************************
0.9315847994370163


In [36]:
xd = pd.read_csv("val.csv")
xd

,browser,os,locale,user_id,gender,location,sites,time,date
0,Firefox,Windows 8,wo-SN,166,f,Singapore/Singapore,"[{'site': 'microsoft.com', 'length': 52}, {'si...",03:40:00,2016-04-29
1,Safari,MacOS,pt-PT,51,m,Australia/Sydney,"[{'site': 'bing.com', 'length': 59}, {'site': ...",22:47:00,2016-08-31
2,Chrome,Debian,pt-BR,113,f,New Zealand/Auckland,"[{'site': 'baidu.com', 'length': 107}, {'site'...",07:06:00,2016-07-10
3,Firefox,Windows 8,wo-SN,160,f,USA/New York,"[{'site': 'vimeo.com', 'length': 100}, {'site'...",21:44:00,2016-01-15
4,Chrome,Windows 8,de-DE,23,m,Germany/Berlin,"[{'site': 'toptal.com', 'length': 60}, {'site'...",01:46:00,2016-02-28
...,...,...,...,...,...,...,...,...,...
8995,Firefox,Windows 10,xh-ZA,124,f,Spain/Madrid,"[{'site': 'jboss.org', 'length': 214}]",07:13:00,2017-09-12
8996,Firefox,Windows 8,uk-UA,118,f,Brazil/Rio de Janeiro,"[{'site': 'mangafox.me', 'length': 106}, {'sit...",15:00:00,2016-03-03
8997,Chrome,Windows 8,en-AU,37,m,New Zealand/Auckland,"[{'site': 'twitter.com', 'length': 109}, {'sit...",21:54:00,2016-11-28
8998,Firefox,Windows 8,ur-PK,36,m,UK/London,"[{'site': 'disqus.com', 'length': 80}, {'site'...",10:44:00,2017-04-22


In [2]:
from src.preprocessing import unwrap_sites

In [3]:
from xgboost import DMatrix


Testing enc and model on validation set


In [2]:
enc = pickle.load(open("encoder.pkl", 'rb'))
model =  pickle.load(open("model.pkl", 'rb'))
val = pickle.load(open("val.pkl", 'rb'))

In [27]:
val = preprocess(val)

In [29]:
val.drop(["date_hour", "time"], axis=1, inplace=True)
cat_cols = val.select_dtypes(include=["object"]).columns


In [30]:
val[cat_cols] = enc.transform(val[cat_cols])

In [33]:
y_val = val.pop("is_joe")
x_val = val

In [35]:
dval = x_val


In [36]:
preds = model.predict(dval)
preds.sum()


41